In [ ]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

In [16]:
import kotlinx.serialization.Serializable

// from previous notebook
@Serializable
data class InterimResult2(
    val cveId: String,
    val ghsaId: String,
    val packageName: String,
    val vulnIntroduced: String,
    val latestVulnVersion: String,
    val fixedVersion: String,
    val fixCommitRepo: String,
    val fixCommitHash: String,
    val jarBefore: String,
    val jarAfter: String,
    val srcBefore: String,
    val srcAfter: String,
    val classFileBefore: String,
    val classFileAfter: String,
    val changedClass: String,
    val changedMethods: String,
    val packageIdBefore: String,
    val packageIdAfter: String,
    val diffParsedJson: String,
    val nloc: Int,
)

In [21]:
val data = DataFrame.readJson(dataDir.resolve("proc/changes.json"))

In [ ]:
import com.google.gson.GsonBuilder

data.rows().shuffled().take(1).map {
    val jsonString = it.diffParsedJson
    DISPLAY(jsonString)
    val gson = GsonBuilder().setPrettyPrinting().create()
    val json = gson.fromJson(jsonString, JsonElement::class.java)
    val niceJsonString = gson.toJson(json)
    DISPLAY(niceJsonString)
}

In [30]:
data class InterimResult3(
    val cveId: String,
    val ghsaId: String,
    val groupArtifact: String,
    val vulnIntroduced: String,
    val latestVulnVersion: String,
    val fixedVersion: String,
    val fixCommitRepo: String,
    val fixCommitHash: String,
    val jarBefore: String,
    val jarAfter: String,
    val srcBefore: String,
    val srcAfter: String,
    val classFileBefore: String,
    val classFileAfter: String,
    val changedClass: String,
    val changedMethods: String,
    val packageIdBefore: String,
    val packageIdAfter: String,
    val diffJson: JsonElement,
    val nloc: Int,
)

val newData = data
    .rows()
    .map {
        val gson = GsonBuilder().setPrettyPrinting().create()
        val json = gson.fromJson(it.diffParsedJson, JsonElement::class.java)
        InterimResult3(
            cveId = it.cveId,
            ghsaId = it.ghsaId,
            groupArtifact = it.packageName,
            vulnIntroduced = it.vulnIntroduced,
            latestVulnVersion = it.latestVulnVersion,
            fixedVersion = it.fixedVersion,
            fixCommitRepo = it.fixCommitRepo,
            fixCommitHash = it.fixCommitHash,
            jarBefore = it.jarBefore,
            jarAfter = it.jarAfter,
            srcBefore = it.srcBefore,
            srcAfter = it.srcAfter,
            classFileBefore = it.classFileBefore,
            classFileAfter = it.classFileAfter,
            changedClass = it.changedClass,
            changedMethods = it.changedMethods,
            packageIdBefore = it.packageIdBefore,
            packageIdAfter = it.packageIdAfter,
            diffJson = json,
            nloc = it.nloc,
        )
    }

In [39]:
import io.ktor.util.*
import java.io.FileOutputStream
import java.io.OutputStreamWriter
import java.nio.charset.Charset

val gson = GsonBuilder().setPrettyPrinting().disableHtmlEscaping().create() 
val json = gson.toJson(newData)
dataDir.resolve("proc/changes.v2.json").writeText(json)